In [3]:
from backend.attMat import attMat, kMer
import numpy as np
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy import stats
from Bio import SeqIO

In [ ]:
# Load attention scores and DNA sequences
expl_scores = pd.read_csv('examples/DNABERTexpl.csv')
expl_scores = expl_scores[]